In [3]:
# ================================
# 1. Data Retrieval and Preparation
# ================================

import os
from dotenv import load_dotenv
from tvDatafeed import TvDatafeedLive, Interval
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.signal import savgol_filter
from sklearn.metrics import mean_squared_error
import pmdarima as pm
import os

# Load environment variables from .env file
load_dotenv()

# Retrieve TradingView credentials from environment variables
username = os.environ.get("TRADINGVIEW_USERNAME")
password = os.environ.get("TRADINGVIEW_PASSWORD")

# Initialize TradingView live data feed
tv = TvDatafeedLive(username, password)

# Fetch historical data for XAUUSD (2000 hourly bars)
prices = tv.get_hist(symbol='XAUUSD', exchange='ICMARKETS',
                     interval=Interval.in_1_hour, n_bars=2000)

# Ensure the 'close' column is in numeric format
prices['close'] = pd.to_numeric(prices['close'], errors='coerce')

# Drop any rows with NaN values in 'close'
prices.dropna(subset=['close'], inplace=True)

# Reset index if necessary
prices.reset_index(drop=True, inplace=True)

# Verify total data points after cleaning
total_data_points = prices.shape[0]
print(f"Total data points after cleaning: {total_data_points}")

# ================================
# 2. Hyperparameter Optimization
# ================================

# Define error metric: Root Mean Squared Error (RMSE)
def calculate_rmse(actual, predicted):
    return np.sqrt(mean_squared_error(actual, predicted))

# Fixed parameters
fixed_train_size = 1200       # Fixed training size
fixed_seasonal = False        # Fixed seasonality
forecast_elements = 24         # Number of data points to forecast (24 hours)
shift_step = 10                # Shift each run by 10 data points
window_length = 19             # Fixed window_length for Savitzky-Golay filter
polyorder = 1                  # Fixed polyorder for Savitzky-Golay filter

# Optimization variables
seasonal_data_lengths = [120, 240, 360, 480, 600, 720]
num_harmonics_list = [10, 20, 30, 40, 50, 60, 70, 80]

# Create a directory to save plots
import os
plot_dir = "forecast_plots"
if not os.path.exists(plot_dir):
    os.makedirs(plot_dir)

# Initialize a list to store results
results = []

# Loop through each combination of seasonal_data_length and num_harmonics
for seasonal_data_length in seasonal_data_lengths:
    for num_harmonics in num_harmonics_list:
        rmse_list = []
        print(f"\nOptimizing for seasonal_data_length={seasonal_data_length}, num_harmonics={num_harmonics}...")
        
        for run in range(num_runs):
            # Define shift for rolling window
            shift = run * shift_step

            # Calculate indices for slicing using positive indices
            start_idx = total_data_points - (fixed_train_size + forecast_elements + shift)
            end_train_idx = total_data_points - (forecast_elements + shift)
            test_start_idx = end_train_idx
            test_end_idx = end_train_idx + forecast_elements

            # Ensure indices are within bounds
            if start_idx < 0 or test_end_idx > total_data_points:
                print(f"  Skipping run {run} due to insufficient data (shift={shift}).")
                continue

            # Slice the data for training and testing
            train_prices = prices.iloc[start_idx:end_train_idx].copy()
            test_prices = prices.iloc[test_start_idx:test_end_idx].copy()

            # Verify that slicing has the correct number of data points
            if len(train_prices) != fixed_train_size or len(test_prices) != forecast_elements:
                print(f"  Run {run} failed: Incorrect slice sizes (train_size={len(train_prices)}, test_size={len(test_prices)}).")
                rmse_list.append(np.nan)
                continue

            # Extract the trend component using Savitzky-Golay filter on training data
            try:
                trend_train = savgol_filter(train_prices['close'], window_length=window_length, polyorder=polyorder)
                # Subtract trend from original data to get the seasonal component
                seasonal_train = train_prices['close'].values - trend_train
            except ValueError as e:
                print(f"  Run {run} failed during Savitzky-Golay filter on training data: {e}")
                rmse_list.append(np.nan)
                continue

            # Fit Auto-ARIMA model on the trend component
            try:
                smodel = pm.auto_arima(
                    trend_train,
                    start_p=1, start_q=1,
                    max_p=3, max_q=3,
                    seasonal=fixed_seasonal,
                    m=24 if fixed_seasonal else 1,    # Set m=24 for daily seasonality if seasonal=True
                    d=None,
                    D=1 if fixed_seasonal else 0,
                    trace=False,
                    error_action='ignore',
                    suppress_warnings=True,
                    stepwise=True
                )

                # Forecast the next 24 data points (trend_forecast)
                forecast_trend = smodel.predict(n_periods=forecast_elements)
            except Exception as e:
                print(f"  Run {run} failed during Auto-ARIMA fitting or forecasting: {e}")
                rmse_list.append(np.nan)
                continue

            # Seasonal Component Forecasting using FFT
            try:
                # Determine the starting index for the seasonal data
                if seasonal_data_length > len(seasonal_train):
                    print(f"  Run {run} failed: seasonal_data_length={seasonal_data_length} exceeds available data.")
                    rmse_list.append(np.nan)
                    continue

                # Extract the last 'seasonal_data_length' points for FFT
                seasonal_data_for_fft = seasonal_train[-seasonal_data_length:]

                # Time indices for the existing data
                N_seasonal = len(seasonal_data_for_fft)
                total_length = N_seasonal + forecast_elements
                extended_indices = np.arange(total_length)

                # Perform FFT on the seasonal data
                fft_result = np.fft.fft(seasonal_data_for_fft)
                fft_freq = np.fft.fftfreq(N_seasonal)

                # Only need positive frequencies
                positive_freq_indices = fft_freq >= 0
                fft_freq_positive = fft_freq[positive_freq_indices]
                fft_result_positive = fft_result[positive_freq_indices]

                # Retain only the first 'num_harmonics' harmonics
                if num_harmonics > len(fft_result_positive):
                    print(f"  Run {run} failed: num_harmonics={num_harmonics} exceeds available harmonics ({len(fft_result_positive)}).")
                    rmse_list.append(np.nan)
                    continue

                frequencies = fft_freq_positive[:num_harmonics]
                amplitudes = fft_result_positive[:num_harmonics]

                # Reconstruct the seasonal signal with forecast
                reconstructed_seasonal = np.zeros(total_length)

                for i in range(num_harmonics):
                    frequency = frequencies[i]
                    amplitude = amplitudes[i]
                    omega = 2 * np.pi * frequency
                    phase = np.angle(amplitude)
                    if frequency == 0:
                        # DC component
                        reconstructed_seasonal += amplitude.real / N_seasonal
                    else:
                        # Harmonic components
                        reconstructed_seasonal += (2 * np.abs(amplitude) / N_seasonal) * np.cos(omega * extended_indices + phase)

                # Extract the forecasted seasonal component
                seasonal_forecast = reconstructed_seasonal[-forecast_elements:]
            except Exception as e:
                print(f"  Run {run} failed during FFT processing: {e}")
                rmse_list.append(np.nan)
                continue

            # Combine the trend and seasonal forecasts
            total_forecast = forecast_trend + seasonal_forecast

            # Actual test data
            actual_values = test_prices['close'].values

            # Calculate RMSE between total forecast and actual test data
            rmse = calculate_rmse(actual_values, total_forecast)
            rmse_list.append(rmse)

            # ================================
            # 4. Visualization
            # ================================

            try:
                # Define the number of training points to plot
                plot_train_size = 240

                # Ensure that the last 'plot_train_size' data points are available
                if plot_train_size > len(train_prices):
                    plot_train_size = len(train_prices)

                # Extract the last 'plot_train_size' training data points
                plot_train_prices = train_prices.iloc[-plot_train_size:].copy()
                plot_test_prices = test_prices.copy()

                # Extract the corresponding trend
                plot_trend_train = trend_train[-plot_train_size:]
                plot_trend_forecast = forecast_trend

                # Extract the corresponding seasonal component
                plot_seasonal_train = seasonal_train[-plot_train_size:]
                plot_seasonal_forecast = seasonal_forecast

                # Combine train and test for plotting
                combined_plot_prices = np.concatenate([plot_train_prices['close'].values, plot_test_prices['close'].values])
                combined_trend = np.concatenate([plot_trend_train, plot_trend_forecast])
                combined_seasonal = np.concatenate([plot_seasonal_train, plot_seasonal_forecast])

                # Create a time index for plotting
                time_train = np.arange(-plot_train_size, 0)
                time_forecast = np.arange(0, forecast_elements)
                time_combined = np.concatenate([time_train, time_forecast])

                # Plotting
                plt.figure(figsize=(12, 6))
                plt.plot(time_combined, combined_plot_prices, label='Original Price', color='blue')
                plt.plot(time_combined, combined_trend, label='Trend', color='red')
                plt.plot(time_combined, combined_trend+combined_seasonal, label='Trend+Seasonal', color='green')
                plt.axvline(x=0, color='black', linestyle='--', label='Forecast Start')
                plt.title(f'Run {run} - Seasonal Data Length: {seasonal_data_length}, Harmonics: {num_harmonics}')
                plt.xlabel('Time (hours)')
                plt.ylabel('Price')
                plt.legend()
                plt.tight_layout()

                # Save the plot
                plot_filename = f"seasonal_length_{seasonal_data_length}_harmonics_{num_harmonics}_run_{run}.png"
                plt.savefig(os.path.join(plot_dir, plot_filename))
                plt.close()

            except Exception as e:
                print(f"  Run {run} failed during plotting: {e}")
                continue

        # Calculate average RMSE across all runs, ignoring NaN values
        if rmse_list:
            avg_rmse = np.nanmean(rmse_list)
        else:
            avg_rmse = np.nan

        # Store the results
        results.append({
            'seasonal_data_length': seasonal_data_length,
            'num_harmonics': num_harmonics,
            'avg_rmse': avg_rmse
        })

        print(f"  Completed: seasonal_data_length={seasonal_data_length}, num_harmonics={num_harmonics}, avg_rmse={avg_rmse:.4f}")

# ================================
# 3. Visualization of Results
# ================================

# Convert results to DataFrame for analysis
results_df = pd.DataFrame(results)

# Display the results
print("\nOptimization Results:")
print(results_df)

# Pivot the DataFrame for heatmap
pivot_table = results_df.pivot('seasonal_data_length', 'num_harmonics', 'avg_rmse')

# Set plot style
sns.set(style="whitegrid")

# Plot a heatmap of the results
plt.figure(figsize=(16, 10))
sns.heatmap(pivot_table, annot=True, fmt=".4f", cmap='viridis')
plt.title('Average RMSE for Different Seasonal Data Lengths and Number of Harmonics')
plt.xlabel('Number of Harmonics')
plt.ylabel('Seasonal Data Length')
plt.show()


ERROR:tvDatafeed.main:error while signin


Total data points after cleaning: 2000

Optimizing for seasonal_data_length=120, num_harmonics=10...
  Completed: seasonal_data_length=120, num_harmonics=10, avg_rmse=12.7099

Optimizing for seasonal_data_length=120, num_harmonics=20...
  Completed: seasonal_data_length=120, num_harmonics=20, avg_rmse=12.9810

Optimizing for seasonal_data_length=120, num_harmonics=30...
  Completed: seasonal_data_length=120, num_harmonics=30, avg_rmse=13.0977

Optimizing for seasonal_data_length=120, num_harmonics=40...
  Completed: seasonal_data_length=120, num_harmonics=40, avg_rmse=13.2236

Optimizing for seasonal_data_length=120, num_harmonics=50...
  Completed: seasonal_data_length=120, num_harmonics=50, avg_rmse=13.2672

Optimizing for seasonal_data_length=120, num_harmonics=60...
  Completed: seasonal_data_length=120, num_harmonics=60, avg_rmse=13.3588

Optimizing for seasonal_data_length=120, num_harmonics=70...
  Run 0 failed: num_harmonics=70 exceeds available harmonics (60).
  Run 1 failed: 

/var/folders/t7/qhsh0qws38d4057943350r400000gn/T/ipykernel_36273/4043863208.py:260: RuntimeWarning: Mean of empty slice
  avg_rmse = np.nanmean(rmse_list)


  Run 0 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 1 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 2 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 3 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 4 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 5 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 6 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 7 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 8 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 9 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 10 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 11 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 12 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 13 failed: num_harmonics=80 exceeds available harmonics (60).
  Run 14 failed: num_harmonics=80 exceeds available harmon

/var/folders/t7/qhsh0qws38d4057943350r400000gn/T/ipykernel_36273/4043863208.py:260: RuntimeWarning: Mean of empty slice
  avg_rmse = np.nanmean(rmse_list)


  Completed: seasonal_data_length=240, num_harmonics=10, avg_rmse=12.1247

Optimizing for seasonal_data_length=240, num_harmonics=20...
  Completed: seasonal_data_length=240, num_harmonics=20, avg_rmse=12.2944

Optimizing for seasonal_data_length=240, num_harmonics=30...
  Completed: seasonal_data_length=240, num_harmonics=30, avg_rmse=12.6409

Optimizing for seasonal_data_length=240, num_harmonics=40...
  Completed: seasonal_data_length=240, num_harmonics=40, avg_rmse=12.6929

Optimizing for seasonal_data_length=240, num_harmonics=50...
  Completed: seasonal_data_length=240, num_harmonics=50, avg_rmse=12.7735

Optimizing for seasonal_data_length=240, num_harmonics=60...
  Completed: seasonal_data_length=240, num_harmonics=60, avg_rmse=12.8563

Optimizing for seasonal_data_length=240, num_harmonics=70...
  Completed: seasonal_data_length=240, num_harmonics=70, avg_rmse=12.9235

Optimizing for seasonal_data_length=240, num_harmonics=80...
  Completed: seasonal_data_length=240, num_harmo

TypeError: DataFrame.pivot() takes 1 positional argument but 4 were given

In [4]:
results_df

,seasonal_data_length,num_harmonics,avg_rmse
0,120,10,12.709860
1,120,20,12.981041
2,120,30,13.097650
3,120,40,13.223626
4,120,50,13.267170
5,120,60,13.358807
6,120,70,NaN
7,120,80,NaN
8,240,10,12.124704
9,240,20,12.294431
